In [ ]:
# castorama.py

#Фото со страницы получилось скачать только одно, правда, большое) Все остальные
# ссылки на странице товара,ведущие на фотографии, в браузере выглядят нормально, а в scrapy ссылка превращается
# в нечто вида "data:image/png base64 , набор символов". Это каcается всех ссылок на фото с тегом
#src. Единственная работающая ссылка такого тега не имеет. Какие пути в scrapy не подставляй,
#если в конце этого пути будет src - ссылка не обрабатывается((. Возможно, сайт блокирует?

import scrapy
from scrapy.http import HtmlResponse
from cs.items import CsparserItem
from scrapy.loader import ItemLoader


class CsSpider(scrapy.Spider):
    name = 'castorama'
    allowed_domains = ['castorama.ru']


    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.start_urls = [f"https://www.castorama.ru/catalogsearch/result/?q={kwargs.get('query')}"]

    def parse(self, response: HtmlResponse):
        next_page = response.xpath('//a[@class="product-list-show-more__link js-product-list-show-more"]/@href').get()
        if next_page:
            yield response.follow(next_page, callback=self.parse)
        links = response.xpath("//a[@class='product-card__img-link']")
        for link in links:
            yield response.follow(link, callback=self.parse_ads)


    def parse_ads(self, response: HtmlResponse):
        loader = ItemLoader(item=CsparserItem(), response=response)
        loader.add_xpath('name', '//h1[@itemprop="name"]/text()')
        loader.add_xpath('price', "//span[@class='price']/span/span/text()")
        loader.add_xpath('photos', '//span[@itemprop = "image"]/@content')
        loader.add_xpath('specs', '//div//*[contains(@class, "specs-table__attribute")]/text()')
        loader.add_value('url', response.url)
        yield loader.load_item()

In [ ]:
# items.py

import scrapy
from itemloaders.processors import MapCompose, TakeFirst


def clear_price(value):
    if value:
        value = value.replace(' ','')
        try:
            value = int(value)
        except:
            return value
        return value

def clear_specs(value):
    value = [''.join(el.replace('\n', ' ') for el in value)]
    value = [el.strip() for el in value if el.isspace() == False]
    #Хотел сделать список более читабельным, соединив в одной строке признак и его значение,
    #попарно объединив элементы в списке. Однако почему-то ни одна из команд под комментарием
    #в mapcompose не работает, хотя если их применить независимо, они дают нужный результат.
    #В чем же может быть дело?

    #value = [value[i] + ' - ' + value[i + 1] for i in range(0, len(value), 2)]

    #si = iter(value)
    #value = list(map(' - '.join, zip(si, si)))
    list(range(0, len(value), 2))
    return value


class CsparserItem(scrapy.Item):
    # define the fields for your item here like:
    name = scrapy.Field(output_processor=TakeFirst())
    price = scrapy.Field(input_processor=MapCompose(clear_price), output_processor=TakeFirst())
    url = scrapy.Field(output_processor=TakeFirst())
    photos = scrapy.Field()
    specs = scrapy.Field(input_processor=MapCompose(clear_specs))
    _id = scrapy.Field()

In [ ]:
# pipelines.py

# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://docs.scrapy.org/en/latest/topics/item-pipeline.html


# useful for handling different item types with a single interface
import scrapy
from itemadapter import ItemAdapter
from scrapy.pipelines.images import ImagesPipeline

class CsparserPipeline:
     def process_item(self, item, spider):
         return item


class CsPhotosPipeline(ImagesPipeline):
    def get_media_requests(self, item, info):
        if item['photos']:
            for img in item['photos']:
                try:
                    yield scrapy.Request(img)
                except Exception as e:
                    print(e)

    def item_completed(self, results, item, info):
        item['photos'] = [itm[1] for itm in results if itm[0]]
        return item

In [ ]:
# runner.py

from twisted.internet import reactor
from scrapy.crawler import CrawlerRunner
from scrapy.utils.log import configure_logging
from scrapy.utils.project import get_project_settings

from cs.spiders.castorama import CsSpider

if __name__ == '__main__':
    configure_logging()
    settings = get_project_settings()
    runner = CrawlerRunner(settings)

    runner.crawl(CsSpider, query='стул')

    reactor.run()


In [ ]:
# settings.py

# Scrapy settings for avitoparser project
#
# For simplicity, this file contains only settings considered important or
# commonly used. You can find more settings consulting the documentation:
#
#     https://docs.scrapy.org/en/latest/topics/settings.html
#     https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
#     https://docs.scrapy.org/en/latest/topics/spider-middleware.html

BOT_NAME = 'castorama'

IMAGES_STORE = 'photos'
#IMAGES_THUMBS = {
   #'medium': (55, 35)
#}


SPIDER_MODULES = ['cs.spiders']
NEWSPIDER_MODULE = 'cs.spiders'

# Crawl responsibly by identifying yourself (and your website) on the user-agent
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0'

# Obey robots.txt rules
ROBOTSTXT_OBEY = False

# Configure maximum concurrent requests performed by Scrapy (default: 16)
CONCURRENT_REQUESTS = 8

LOG_ENABLED = True
LOG_LEVEL = "DEBUG"

# Configure a delay for requests for the same website (default: 0)
# See https://docs.scrapy.org/en/latest/topics/settings.html#download-delay
# See also autothrottle settings and docs
DOWNLOAD_DELAY = 1.5
# The download delay setting will honor only one of:
#CONCURRENT_REQUESTS_PER_DOMAIN = 16
#CONCURRENT_REQUESTS_PER_IP = 16

# Disable cookies (enabled by default)
COOKIES_ENABLED = True

# Disable Telnet Console (enabled by default)
#TELNETCONSOLE_ENABLED = False

# Override the default request headers:
# DEFAULT_REQUEST_HEADERS = {
#   'cookie':'yandexuid=1259160151646070856; yuidss=1259160151646070856; yabs-sid=1937487681646070856; ymex=1961430856.yrts.1646070856#1961430856.yrtsi.1646070856; gdpr=0; _ym_uid=1646070893156269093; my=YwA=; amcuid=8340317491646422388; yandex_login=Orleon.ya; i=wdTJiryHb+LLnT5mcRSlLhkbNWoRkyWlPsQzRYQZNAx4MhNo6TM/BV8R51m0d0jIkpBJv0xmNtTmxeXkiTiWht9wDgw=; is_gdpr=0; is_gdpr_b=CNaZZBDuaigC; yandex_gid=44; cycada=Pu5H3ElFTWHQAAU7lrMwAL/hvOtk0vJNciYyzRBtN1E=; _ym_d=1649519883; yabs-frequency=/5/0000000000000000/LVTwO9j8Vb84HY6q2tDmb00000H68M-bN1mQii9h14Od8krF10oancu4HYC0/; yp=1665287886.szm.1_25:2560x1440:1990x1072#1962472582.udn.cDpPcmxlb24ueWE%3D#1651771282.ygu.1#1651771283.spcs.l#1651857692.csc.1; ys=udn.cDpPcmxlb24ueWE%3D#c_chck.8048816; Session_id=3:1649696374.5.0.1647112582511:GiqSWw:25.1.2:1|271833109.0.2|3:250771.178521.H07E8OxfqD1kCKQ9zmuC7qex_qY; sessionid2=3:1649696374.5.0.1647112582511:GiqSWw:25.1.2:1|271833109.0.2|3:250771.178521.H07E8OxfqD1kCKQ9zmuC7qex_qY'
# }

# Enable or disable spider middlewares
# See https://docs.scrapy.org/en/latest/topics/spider-middleware.html
#SPIDER_MIDDLEWARES = {
#    'avitoparser.middlewares.AvitoparserSpiderMiddleware': 543,
#}

# Enable or disable downloader middlewares
# See https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
#DOWNLOADER_MIDDLEWARES = {
#    'avitoparser.middlewares.AvitoparserDownloaderMiddleware': 543,
#}

# Enable or disable extensions
# See https://docs.scrapy.org/en/latest/topics/extensions.html
#EXTENSIONS = {
#    'scrapy.extensions.telnet.TelnetConsole': None,
#}

# Configure item pipelines
# See https://docs.scrapy.org/en/latest/topics/item-pipeline.html
ITEM_PIPELINES = {
   'cs.pipelines.CsparserPipeline': 300,
   'cs.pipelines.CsPhotosPipeline': 200
}

# Enable and configure the AutoThrottle extension (disabled by default)
# See https://docs.scrapy.org/en/latest/topics/autothrottle.html
#AUTOTHROTTLE_ENABLED = True
# The initial download delay
#AUTOTHROTTLE_START_DELAY = 5
# The maximum download delay to be set in case of high latencies
#AUTOTHROTTLE_MAX_DELAY = 60
# The average number of requests Scrapy should be sending in parallel to
# each remote server
#AUTOTHROTTLE_TARGET_CONCURRENCY = 1.0
# Enable showing throttling stats for every response received:
#AUTOTHROTTLE_DEBUG = False

# Enable and configure HTTP caching (disabled by default)
# See https://docs.scrapy.org/en/latest/topics/downloader-middleware.html#httpcache-middleware-settings
#HTTPCACHE_ENABLED = True
#HTTPCACHE_EXPIRATION_SECS = 0
#HTTPCACHE_DIR = 'httpcache'
#HTTPCACHE_IGNORE_HTTP_CODES = []
#HTTPCACHE_STORAGE = 'scrapy.extensions.httpcache.FilesystemCacheStorage'